## Libraries Installation

In [3]:
!pip install pdfplumber


In [18]:
pip install pytesseract Pillow pdfplumber


Note: you may need to restart the kernel to use updated packages.


In [30]:
import re
import pandas as pd

## PDF processing
## Extracting text using :
### *pdfplumber for text-based PDFs.
### *Tesseract OCR for scanned image-based PDFs.

In [20]:
import pdfplumber
import pytesseract
from PIL import Image

pdf_path = "C://Users//javva//OneDrive//Documents//Dummy-Bank-Statement.pdf"

def extract_text_from_pdf(pdf_path):
    all_text = ""
    
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Check if the page has any text
            text = page.extract_text()
            
            if text:  # If the page contains text, use it
                all_text += text
            else:
                # If no text is found, apply OCR to the image of the page
                print("OCR applied to page")
                image = page.to_image()
                ocr_text = pytesseract.image_to_string(image.original)  # Use OCR on the image
                all_text += ocr_text
                
    return all_text

# Extract text from PDF (including OCR if needed)
extracted_text = extract_text_from_pdf(pdf_path)

# Print the extracted text to verify
print(extracted_text) # Print the first 500 characters to inspect the result


Dummy Bank Statement
Bank Name: People's Trust Bank
Customer Name: John A. Doe
Account Number: 123-456-789
Statement Period: July 1, 2023 - July 31, 2023
Address: 123 Maple Street, Anytown, AT 12345
Account Summary
Opening Balance: $5,000.00
Closing Balance: $4,250.00
Transactions
Date Description Withdrawals Deposits Balance
07/01/2023 Opening Balance - - $5,000.00
07/02/2023 Electric Bill Payment $250.00 - $4,750.00
07/05/2023 Grocery Store $150.00 - $4,600.00
07/08/2023 Salary Deposit - $1,000.00 $5,600.00
07/12/2023 Online Shopping - Z-Mart $100.00 - $5,500.00
07/15/2023 Cash Withdrawal - ATM $200.00 - $5,300.00
Copyright @ SampleTemplates.com2
07/18/2023 Car Insurance Premium $350.00 - $4,950.00
07/22/2023 Coffee Shop $20.00 - $4,930.00
07/25/2023 Gas Station $50.00 - $4,880.00
07/28/2023 Water Bill Payment $300.00 - $4,580.00
07/30/2023 Gym Membership Fee $80.00 - $4,500.00
Movie Streaming Service
07/31/2023 $50.00 - $4,450.00
Subscription
07/31/2023 Monthly Maintenance Fee $200.

# Data Preprocessing

## Filtering out transaction lines from the extracted text based on date pattern

In [23]:
# Extract only transaction lines
transaction_lines = []
for line in all_text.split('\n'):
    if re.match(r"^\d{2}/\d{2}/\d{4}", line):
        transaction_lines.append(line)

print("Filtered Transaction Lines:")
for line in transaction_lines:
    print(line)

transaction_text = "\n".join(transaction_lines)

Filtered Transaction Lines:
07/01/2023 Opening Balance - - $5,000.00
07/02/2023 Electric Bill Payment $250.00 - $4,750.00
07/05/2023 Grocery Store $150.00 - $4,600.00
07/08/2023 Salary Deposit - $1,000.00 $5,600.00
07/12/2023 Online Shopping - Z-Mart $100.00 - $5,500.00
07/15/2023 Cash Withdrawal - ATM $200.00 - $5,300.00
07/18/2023 Car Insurance Premium $350.00 - $4,950.00
07/22/2023 Coffee Shop $20.00 - $4,930.00
07/25/2023 Gas Station $50.00 - $4,880.00
07/28/2023 Water Bill Payment $300.00 - $4,580.00
07/30/2023 Gym Membership Fee $80.00 - $4,500.00
07/31/2023 $50.00 - $4,450.00
07/31/2023 Monthly Maintenance Fee $200.00 - $4,250.00


## Defining a regular expression (regex) to extract transaction details 

In [28]:
# Define a refined regex pattern to extract transactions
pattern = r"(\d{2}/\d{2}/\d{4}|\d{2}-\d{2}-\d{4}|\d{4}/\d{2}/\d{2})\s+(.+?)\s+\$?(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)?\s*-?\$?(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)?\s+\$?(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)"

matches = re.findall(pattern, transaction_text)
print("Matches found:", matches)  # Debugging line to check matches

Matches found: [('07/01/2023', 'Opening Balance -', '', '', '5,000.00'), ('07/02/2023', 'Electric Bill Payment', '250.00', '', '4,750.00'), ('07/05/2023', 'Grocery Store', '150.00', '', '4,600.00'), ('07/08/2023', 'Salary Deposit', '', '', '1,000.00'), ('07/12/2023', 'Online Shopping - Z-Mart', '100.00', '', '5,500.00'), ('07/15/2023', 'Cash Withdrawal - ATM', '200.00', '', '5,300.00'), ('07/18/2023', 'Car Insurance Premium', '350.00', '', '4,950.00'), ('07/22/2023', 'Coffee Shop', '20.00', '', '4,930.00'), ('07/25/2023', 'Gas Station', '50.00', '', '4,880.00'), ('07/28/2023', 'Water Bill Payment', '300.00', '', '4,580.00'), ('07/30/2023', 'Gym Membership Fee', '80.00', '', '4,500.00'), ('07/31/2023', '$50.00', '', '', '4,450.00'), ('07/31/2023', 'Monthly Maintenance Fee', '200.00', '', '4,250.00')]


## Matching data extracted from the regex and organizes them into a list of dictionaries

In [25]:
# Convert matches to a list of dictionaries
transactions = []
for match in matches:
    date, description, withdrawal, deposit, balance = match
    amount = float(withdrawal.replace(',', '')) if withdrawal else -float(deposit.replace(',', ''))
    transactions.append({
        "date": date,
        "description": description.strip(),
        "amount": amount,
        "balance": float(balance.replace(',', ''))
    })

print("Extracted transactions:")
for transaction in transactions:
    print(transaction)

Extracted transactions:
{'date': '07/02/2023', 'description': 'Electric Bill Payment', 'amount': 250.0, 'balance': 4750.0}
{'date': '07/05/2023', 'description': 'Grocery Store', 'amount': 150.0, 'balance': 4600.0}
{'date': '07/12/2023', 'description': 'Online Shopping - Z-Mart', 'amount': 100.0, 'balance': 5500.0}
{'date': '07/15/2023', 'description': 'Cash Withdrawal - ATM', 'amount': 200.0, 'balance': 5300.0}
{'date': '07/18/2023', 'description': 'Car Insurance Premium', 'amount': 350.0, 'balance': 4950.0}
{'date': '07/22/2023', 'description': 'Coffee Shop', 'amount': 20.0, 'balance': 4930.0}
{'date': '07/25/2023', 'description': 'Gas Station', 'amount': 50.0, 'balance': 4880.0}
{'date': '07/28/2023', 'description': 'Water Bill Payment', 'amount': 300.0, 'balance': 4580.0}
{'date': '07/30/2023', 'description': 'Gym Membership Fee', 'amount': 80.0, 'balance': 4500.0}
{'date': '07/31/2023', 'description': 'Monthly Maintenance Fee', 'amount': 200.0, 'balance': 4250.0}


 # Categorizing Text
 ##  Categorizing transaction descriptions into predefined categories and then storing the results in a DataFrame

In [34]:
# Define categories and keywords
categories = {
    "Food": ["grocery", "coffee", "restaurant"],
    "Travel": ["uber", "lyft", "bus", "trip", "gas station"],
    "Utilities": ["electric", "water", "insurance"],
    "Subscription": ["salary", "streaming"],
    "Payment": ["maintenance", "atm"]
}

# Function to categorize a transaction
def categorize_transaction(description):
    for category, keywords in categories.items():
        if any(keyword in description.lower() for keyword in keywords):
            return category
    return "Other"
    
# Categorize transactions
for transaction in transactions:
    transaction["category"] = categorize_transaction(transaction["description"])

# Convert to DataFrame for better handling
df = pd.DataFrame(transactions)
print("DataFrame:")
print(df)  # Debugging line to check DataFrame

Categorized Transactions:
{'date': '07/02/2023', 'description': 'Electric Bill Payment', 'amount': 250.0, 'balance': 4750.0, 'category': 'Utilities'}
{'date': '07/05/2023', 'description': 'Grocery Store', 'amount': 150.0, 'balance': 4600.0, 'category': 'Food'}
{'date': '07/12/2023', 'description': 'Online Shopping - Z-Mart', 'amount': 100.0, 'balance': 5500.0, 'category': 'Other'}
{'date': '07/15/2023', 'description': 'Cash Withdrawal - ATM', 'amount': 200.0, 'balance': 5300.0, 'category': 'Payment'}
{'date': '07/18/2023', 'description': 'Car Insurance Premium', 'amount': 350.0, 'balance': 4950.0, 'category': 'Utilities'}
{'date': '07/22/2023', 'description': 'Coffee Shop', 'amount': 20.0, 'balance': 4930.0, 'category': 'Food'}
{'date': '07/25/2023', 'description': 'Gas Station', 'amount': 50.0, 'balance': 4880.0, 'category': 'Travel'}
{'date': '07/28/2023', 'description': 'Water Bill Payment', 'amount': 300.0, 'balance': 4580.0, 'category': 'Utilities'}
{'date': '07/30/2023', 'descrip

## Grouping the transactions by their category and then calculating the total amount for each category

In [35]:
# Group by category and calculate the sum of amounts
if not df.empty:
    grouped = df.groupby("category").sum(numeric_only=True)["amount"]
    print("Grouped totals:")
    print(grouped)  # Debugging line to check grouped totals
else:
    print("No transactions found.")

Grouped totals:
category
Food         170.0
Other        180.0
Payment      400.0
Travel        50.0
Utilities    900.0
Name: amount, dtype: float64


## print the categories for reference

In [6]:
from fpdf import FPDF, XPos, YPos

# Create a PDF class
class PDF(FPDF):
    def header(self):
        self.set_font('Helvetica', 'B', 12)
        self.cell(0, 10, 'Categorized Transactions', 0, new_x=XPos.LMARGIN, new_y=YPos.NEXT, align='C')

    def footer(self):
        self.set_y(-15)
        self.set_font('Helvetica', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, new_x=XPos.RIGHT, new_y=YPos.TOP, align='C')

    def chapter_title(self, category):
        self.set_font('Helvetica', 'B', 12)
        self.cell(0, 10, category, 0, new_x=XPos.LMARGIN, new_y=YPos.NEXT, align='L')
        self.ln(10)

    def chapter_body(self, transactions):
        self.set_font('Helvetica', '', 12)
        for transaction in transactions:
            self.cell(0, 10, f"{transaction['date']} {transaction['description']} {transaction['amount']:.2f}", 0, new_x=XPos.LMARGIN, new_y=YPos.NEXT)

# Create the PDF object
pdf = PDF()
pdf.add_page()

# Add categorized transactions
grouped_transactions = df.groupby("category")
for category, group in grouped_transactions:
    pdf.chapter_title(category)
    pdf.chapter_body(group.to_dict(orient='records'))

# Save the PDF
pdf.output("categorized_transactions.pdf")
